In [1]:
import os
import numpy as np
import pandas as pd
import swifter
from sklearn.model_selection import train_test_split
import utils
import catboost

In [2]:
DATA_PATH = "./data"

In [3]:
test = pd.read_csv("data/test_private_v3_track_1.csv.gz",
                   usecols=[utils.ID_COLUMN] + utils.SIMPLE_FEATURE_COLUMNS,
                   index_col=utils.ID_COLUMN)

/root/anaconda3/envs/idao_cb/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
test.columns

Index(['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]', 'avg_cs[1]',
       'avg_cs[2]', 'avg_cs[3]', 'ndof', 'MatchedHit_TYPE[0]',
       'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]', 'Lextra_

In [5]:
func = utils.find_closest_hit_per_station

In [6]:
dt_test  = pd.read_hdf('closest_hits_features.test.filled.m9999.v1.private.hdf', 'key')

In [7]:
dt_test.columns  = [func.__name__ + '_' + str(c) if str(c).isnumeric() else c for c in dt_test.columns]

In [8]:
test = pd.concat([test, dt_test], axis=1, copy=False)

In [9]:
test.shape

(1452188, 113)

In [10]:
train = test

## Новые признаки

In [11]:
NA_VALUE = -9999.0

In [12]:
def fillna(df):
    """
    Пропуски только в 2 и 3 колонках
    """
    
    for axis in 'XYZ':
        cols = ['MatchedHit_{}[{}]'.format(axis, i) for i in range(4)]
        mask = np.isclose(df.loc[:, cols[2]], NA_VALUE)
        mask = mask & ~np.isclose(df.loc[:, cols[3]], NA_VALUE)
        
        df.loc[mask, cols[2]] = df.loc[mask, [cols[1], cols[3]]].mean(axis=1)
    return df

In [13]:
# train, test = map(fillna, [train, test])

In [14]:
train = fillna(train)

### Вычесть среднее из MatchedHit_Z

In [15]:
matched_hit_z_cols = train.columns.str.startswith("MatchedHit_Z")
mask = np.isclose(train.loc[:, matched_hit_z_cols], NA_VALUE).any(axis=0)
matched_hit_z_mean = train.loc[~mask, matched_hit_z_cols].mean(axis=0)
matched_hit_z_mean

MatchedHit_Z[0]    15403.4175
MatchedHit_Z[1]    16609.1150
MatchedHit_Z[2]    17802.2660
MatchedHit_Z[3]    18910.7980
dtype: float64

In [16]:
matched_hit_z_cols = train.columns[train.columns.str.startswith("MatchedHit_Z")]
matched_hit_z_mean = pd.Series([15270, 16470, 17670, 18870], index=matched_hit_z_cols)
matched_hit_z_mean

MatchedHit_Z[0]    15270
MatchedHit_Z[1]    16470
MatchedHit_Z[2]    17670
MatchedHit_Z[3]    18870
dtype: int64

In [17]:
def make_unbiased_z(df):
    for i in range(4):
        distance = np.zeros(df.shape[0])
    
        col = f"MatchedHit_Z[{i}]"
        mask = np.isclose(df.loc[:, col], NA_VALUE)
        distance[~mask] = df.loc[~mask, col] - matched_hit_z_mean[col]
        distance[mask] = NA_VALUE
    
        df[col] = distance
        
        """
        df[f"MatchedHit_Z[{i}]_sign"] = np.sign(df[col]).astype(np.int8)
        """
    return df

In [18]:
# train, test = map(make_unbiased_z, [train, test])

In [19]:
train = make_unbiased_z(train)

In [20]:
train.loc[:, train.columns.str.startswith("MatchedHit_Z")].head()

,MatchedHit_Z[0],MatchedHit_Z[1],MatchedHit_Z[2],MatchedHit_Z[3]
0,136.950,144.400,138.152,147.201
1,129.885,133.830,126.380,-65.605
2,-61.252,-53.578,-59.605,-50.332
3,-71.983,-68.110,-75.630,-67.846
4,-64.852,-60.414,-67.373,-59.027


### Расстояние до центра (+)

In [21]:
def compute_polar_distances_matched_hit(df):
    for i in range(4):
        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
    
        name = f"MatchedHit_R[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.log(1.0 + np.power(df.loc[~mask, cols], 2).sum(axis=1))
        
        name = f"MatchedHit_A[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.arctan2(df.loc[~mask, cols[1]], df.loc[~mask, cols[0]])

    # намеренное переупорядочивание признаков
    cols  = df.columns[df.columns.str.startswith("MatchedHit_R")].tolist()
    cols += df.columns[df.columns.str.startswith("MatchedHit_A")].tolist()
    cols  = df.columns[:-len(cols)].tolist() + cols
    df = df[cols]

    return df

In [22]:
# train, test = map(compute_polar_distances_matched_hit, [train, test])

In [23]:
train = compute_polar_distances_matched_hit(train)

In [24]:
train.loc[:, train.columns.str.startswith("MatchedHit_R")].head()

,MatchedHit_R[0],MatchedHit_R[1],MatchedHit_R[2],MatchedHit_R[3]
0,12.930077,13.078763,13.207914,13.338887
1,16.332379,16.538395,16.718047,16.903495
2,14.370883,14.524747,14.662623,14.794065
3,16.525674,16.678201,16.831457,16.963506
4,13.714256,13.840950,13.964308,14.099488


In [25]:
train.loc[:, train.columns.str.startswith("MatchedHit_A")].head()

,MatchedHit_A[0],MatchedHit_A[1],MatchedHit_A[2],MatchedHit_A[3]
0,1.002782,1.002964,1.010349,1.008445
1,-2.885018,-2.891988,-2.897091,-2.974431
2,1.522447,1.503224,1.531503,1.530494
3,-2.844808,-2.844651,-2.846558,-2.846975
4,2.042894,2.019910,2.000288,2.000319


In [26]:
def compute_polar_distances_lextra(df):
    for i in range(4):
        cols = [f"Lextra_X[{i}]", f"Lextra_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
    
        name = f"Lextra_R[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.log(1.0 + np.power(df.loc[~mask, cols], 2).sum(axis=1))

        """
        name = f"Lextra_A[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.arctan2(df.loc[~mask, cols[1]], df.loc[~mask, cols[0]])
        """
    
    # намеренное переупорядочивание признаков
    cols  = df.columns[df.columns.str.startswith("Lextra_R")].tolist()
    cols += df.columns[df.columns.str.startswith("Lextra_A")].tolist()
    cols  = df.columns[:-len(cols)].tolist() + cols
    df = df[cols]

    return df

In [27]:
# train, test = map(compute_polar_distances_lextra, [train, test])

In [28]:
train = compute_polar_distances_lextra(train)

/root/anaconda3/envs/idao_cb/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/root/anaconda3/envs/idao_cb/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [29]:
train.loc[:, train.columns.str.startswith("Lextra_R")].head()

,Lextra_R[0],Lextra_R[1],Lextra_R[2],Lextra_R[3]
0,12.857646,12.991062,13.116407,13.234573
1,16.273759,16.456256,16.623508,16.777865
2,14.362690,14.515103,14.656819,14.789229
3,16.546279,16.713553,16.867922,17.011233
4,13.777614,13.908398,14.031834,14.148628


In [30]:
train.loc[:, train.columns.str.startswith("Lextra_A")].head()

""
0
1
2
3
4


### Дельты для траекторий MatchedHit_X, MatchedHit_Y

In [31]:
def diff_matched_hit(df):
    for axis in ['X', 'Y']:
        for i, j in zip([0, 1, 2, 3], [1, 2, 3, 0]):
            col_i = f"MatchedHit_{axis}[{i}]"
            col_j = f"MatchedHit_{axis}[{j}]"
            mask = np.isclose(df.loc[:, [col_i, col_j]], NA_VALUE).any(axis=1)

            name = f"MatchedHit_{axis}[{j}/{i}]_delta"
            df[name] = df[name + '_abs'] = NA_VALUE
            
            df.loc[~mask, name] = df.loc[~mask, col_j] - df.loc[~mask, col_i]
            df.loc[~mask, name + '_abs'] = np.abs(df.loc[~mask, name])
    return df

In [32]:
# train, test = map(diff_matched_hit, [train, test])

In [33]:
train = diff_matched_hit(train)

### Дельты и отношения для радиусов (+)

In [34]:
def ratio_radius_matched_hit(df):
    for i, j in zip([0, 1, 2, 3], [1, 2, 3, 0]):
        col_i = f"MatchedHit_R[{i}]"
        col_j = f"MatchedHit_R[{j}]"
        mask = np.isclose(df.loc[:, [col_i, col_j]], NA_VALUE).any(axis=1)

        name = f"MatchedHit_R[{j}/{i}]_delta"
        name_abs = name + '_abs'
        df[name] = df[name_abs] = NA_VALUE
        
        df.loc[~mask, name] = df.loc[~mask, col_j] - df.loc[~mask, col_i]
        df.loc[~mask, name_abs] = np.abs(df.loc[~mask, name])
        
        name = f"MatchedHit_R[{j}/{i}]_frac"
        df[name] = NA_VALUE
        df.loc[~mask, name] = df.loc[~mask, col_j] / df.loc[~mask, col_i]
    return df

In [35]:
# train, test = map(ratio_radius_matched_hit, [train, test])

In [36]:
train = ratio_radius_matched_hit(train)

In [37]:
cols = filter(lambda s: s.startswith("MatchedHit_R") and ("_delta" in s or "_frac" in s), train.columns)
train.loc[:, cols].head()

,MatchedHit_R[1/0]_delta,MatchedHit_R[1/0]_delta_abs,MatchedHit_R[1/0]_frac,MatchedHit_R[2/1]_delta,MatchedHit_R[2/1]_delta_abs,MatchedHit_R[2/1]_frac,MatchedHit_R[3/2]_delta,MatchedHit_R[3/2]_delta_abs,MatchedHit_R[3/2]_frac,MatchedHit_R[0/3]_delta,MatchedHit_R[0/3]_delta_abs,MatchedHit_R[0/3]_frac
0,0.148686,0.148686,1.011499,0.129151,0.129151,1.009875,0.130973,0.130973,1.009916,-0.408810,0.408810,0.969352
1,0.206016,0.206016,1.012614,0.179652,0.179652,1.010863,0.185449,0.185449,1.011093,-0.571117,0.571117,0.966213
2,0.153863,0.153863,1.010707,0.137877,0.137877,1.009493,0.131441,0.131441,1.008964,-0.423181,0.423181,0.971395
3,0.152526,0.152526,1.009230,0.153256,0.153256,1.009189,0.132049,0.132049,1.007845,-0.437831,0.437831,0.974190
4,0.126694,0.126694,1.009238,0.123358,0.123358,1.008913,0.135179,0.135179,1.009680,-0.385231,0.385231,0.972678


In [38]:
def ratio_radius_lextra_hit(df):
    for i, j in zip([0, 1, 2, 3], [1, 2, 3, 0]):
        col_i = f"Lextra_R[{i}]"
        col_j = f"Lextra_R[{j}]"
        mask = np.isclose(df.loc[:, [col_i, col_j]], NA_VALUE).any(axis=1)

        name = f"Lextra_R[{j}/{i}]_delta"
        name_abs = name + '_abs'
        df[name] = df[name_abs] = NA_VALUE
        
        df.loc[~mask, name] = df.loc[~mask, col_j] - df.loc[~mask, col_i]
        df.loc[~mask, name_abs] = np.abs(df.loc[~mask, name])
        
#         name = f"Lextra_R[{j}/{i}]_frac"
#         df[name] = NA_VALUE
#         df.loc[~mask, name] = df.loc[~mask, col_j] / df.loc[~mask, col_i]
    return df

In [39]:
# train, test = map(ratio_radius_lextra_hit, [train, test])

In [40]:
train = ratio_radius_lextra_hit(train)

In [41]:
cols = filter(lambda s: s.startswith("Lextra_R") and ("_delta" in s or "_frac" in s), train.columns)
train.loc[:, cols].head()

,Lextra_R[1/0]_delta,Lextra_R[1/0]_delta_abs,Lextra_R[2/1]_delta,Lextra_R[2/1]_delta_abs,Lextra_R[3/2]_delta,Lextra_R[3/2]_delta_abs,Lextra_R[0/3]_delta,Lextra_R[0/3]_delta_abs
0,0.133416,0.133416,0.125345,0.125345,0.118166,0.118166,-0.376927,0.376927
1,0.182497,0.182497,0.167252,0.167252,0.154357,0.154357,-0.504106,0.504106
2,0.152413,0.152413,0.141715,0.141715,0.132410,0.132410,-0.426539,0.426539
3,0.167274,0.167274,0.154368,0.154368,0.143311,0.143311,-0.464953,0.464953
4,0.130783,0.130783,0.123436,0.123436,0.116794,0.116794,-0.371014,0.371014


### Расстояние между Lextra и MatchedHit (+)

In [42]:
def distance_dist_lextra_matched(df):
    distance_counts = np.zeros(df.shape[0], dtype=int)
    
    distance_se = np.zeros(df.shape[0])
    distance_ae = np.zeros(df.shape[0])
    
    for i in range(4):
        distance = np.zeros(df.shape[0])

        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]", f"Lextra_X[{i}]", f"Lextra_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)

        mse = df.loc[~mask, cols[:2]].values - df.loc[~mask, cols[2:]].values
        mse = np.power(mse, 2.0).sum(axis=1)
        
        mae = df.loc[~mask, cols[:2]].values - df.loc[~mask, cols[2:]].values
        mae = np.abs(mae).max(axis=1)

        distance[~mask] = np.log(1.0 + mse)
        distance[mask] = NA_VALUE
        
        distance_se[~mask] += mse
        distance_ae[~mask] = np.maximum(distance_ae[~mask], mae)
        distance_counts[~mask] += 1

        df[f"distance_LextraMatchedHit[{i}]"] = distance

    distance_se /= distance_counts
    df[f"distance_LextraMatchedHit_SE"] = distance_se
    df[f"distance_LextraMatchedHit_AE"] = distance_ae

    return df

In [43]:
# train, test = map(distance_dist_lextra_matched, [train, test])

In [44]:
train = distance_dist_lextra_matched(train)

In [45]:
train.loc[:, train.columns.str.startswith("distance_LextraMatchedHit")].head()

,distance_LextraMatchedHit[0],distance_LextraMatchedHit[1],distance_LextraMatchedHit[2],distance_LextraMatchedHit[3],distance_LextraMatchedHit_SE,distance_LextraMatchedHit_AE
0,6.787645,6.930203,7.078433,7.381559,1174.431164,34.52456
1,9.315024,10.127164,10.571843,12.132834,65252.293743,344.27400
2,4.357462,6.408753,6.697971,7.287096,738.318816,37.75468
3,8.855256,9.224567,9.392788,9.816331,11870.652337,100.41430
4,6.986883,7.280281,7.445496,6.714575,1266.551534,32.45573


### Абсолютное и относительное смещение

In [46]:
def offset_matched_hit(df):
    offset_abs = np.sqrt(np.power(df.loc[:, ["MatchedHit_X[0]", f"MatchedHit_Y[0]"]], 2).sum(axis=1))
    
    for i, j in zip([0, 1, 2], [1, 2, 3]):
        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]", f"MatchedHit_X[{j}]", f"MatchedHit_Y[{j}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
        
        mse = df.loc[~mask, cols[:2]].values - df.loc[~mask, cols[2:]].values
        mse = np.sqrt(np.power(mse, 2.0).sum(axis=1))
        offset_abs[~mask] += mse
        
    offset_from = df.loc[:, ["MatchedHit_X[0]", f"MatchedHit_Y[0]"]].values
    offset_to = np.zeros(shape=(df.shape[0], 2))
    
    for i in range(4):
        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
        offset_to[~mask] = df.loc[~mask, cols].values
    
    offset_rel = np.sqrt(np.power(offset_from - offset_to, 2.0).sum(axis=1))
    
    df["MatchedHit_offset_abs"] = offset_abs
    df["MatchedHit_offset_rel"] = offset_rel
    df["MatchedHit_offset_rel_div_abs"] = offset_rel / offset_abs
    
    return df

In [47]:
# train, test = map(offset_matched_hit, [train, test])

In [48]:
train = offset_matched_hit(train)

In [49]:
train.loc[:, train.columns.str.startswith("MatchedHit_offset_")].head()

,MatchedHit_offset_abs,MatchedHit_offset_rel,MatchedHit_offset_rel_div_abs
0,788.278385,145.723790,0.184863
1,4809.903874,1218.636228,0.253360
2,1642.616613,311.288521,0.189508
3,4826.011979,948.869716,0.196616
4,1159.708071,206.788736,0.178311


### Отношение дисперсий

In [50]:
def ratio_mextra_matched(df):    
    for i in range(4):
        cols = [f"MatchedHit_DX[{i}]", f"MatchedHit_DY[{i}]", f"Mextra_DX2[{i}]", f"Mextra_DY2[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)

        tmpX = df.loc[~mask, cols[::2]]
        tmpY = df.loc[~mask, cols[1::2]]
        
        ratio = tmpX.iloc[:, 0] / np.sqrt(tmpX.iloc[:, 1]) + \
                tmpY.iloc[:, 0] / np.sqrt(tmpY.iloc[:, 1])

        name = f"ratio_MextraMatchedHit[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = ratio

    return df

In [51]:
# train, test = map(ratio_mextra_matched, [train, test])

In [52]:
train = ratio_mextra_matched(train)

In [53]:
train.loc[:, train.columns.str.startswith("ratio_MextraMatchedHit")].head()

,ratio_MextraMatchedHit[0],ratio_MextraMatchedHit[1],ratio_MextraMatchedHit[2],ratio_MextraMatchedHit[3]
0,8.568041,5.452831,5.923010,4.591995
1,5.836899,3.697998,3.890070,2.942780
2,15.471354,9.812719,10.688360,8.330187
3,9.402639,6.001297,6.386638,4.884344
4,3.857346,2.445352,2.635871,2.025441


### Левые и правые границы для Lextra (- не влезает в память)

In [54]:
def sigma_lextra_hit(df):
    for axis in ['X', 'Y']:
        for i in range(0, 4):
            col_i = f"Lextra_{axis}[{i}]"
            col_j = f"Mextra_D{axis}2[{i}]"
            mask = np.isclose(df.loc[:, [col_i, col_j]], NA_VALUE).any(axis=1)

            for col, factor in zip([f"Lextra_{axis}[{i}]_minus_sigma2",
                                    f"Lextra_{axis}[{i}]_minus_sigma",
                                    f"Lextra_{axis}[{i}]_plus_sigma",
                                    f"Lextra_{axis}[{i}]_plus_sigma2"],
                                   [-2, -1, 1, 2]):
                df[col] = NA_VALUE
                df.loc[~mask, col] = df.loc[~mask, col_i] + factor * np.sqrt(df.loc[~mask, col_j])
        
    return df

### Число объектов в кластерах

In [55]:
def cluster_size(df):
    for i in range(4):
        name = f'cl_size[{i}]'
        df[name] = NA_VALUE
        
        cols = [f'ncl[{i}]', f'avg_cs[{i}]']
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
        df.loc[~mask, name] = df.loc[~mask, cols[0]] * df.loc[~mask, cols[1]]
    return df

In [56]:
# train, test = map(cluster_size, [train, test])

In [57]:
train = cluster_size(train)

In [58]:
train.loc[:, train.columns.str.startswith('cl_size')].head()

,cl_size[0],cl_size[1],cl_size[2],cl_size[3]
0,203.999998,22.999999,21.000000,28.000000
1,180.000000,58.000001,10.000001,16.000001
2,150.999998,27.000000,15.000000,21.000000
3,93.000001,10.000000,22.000000,19.000001
4,225.000009,150.000001,33.000000,13.000000


### Последняя площадка

In [59]:
def station_latest(df):
    station = np.zeros(df.shape[0], dtype=int)
    
    for i in range(4):
        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]", f"MatchedHit_Z[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
        station[~mask] += 1
        
    df["station_latest"] = station
    
    return df

In [60]:
# train, test = map(station_latest, [train, test])

In [61]:
# train.loc[:, "station_latest"].head()

In [62]:
# np.unique(train['station_latest'], return_counts=True)

### Еще один момент

In [63]:
def momentum(df):
    mask = np.isclose(df.loc[:, ['P', 'PT']], NA_VALUE).any(axis=1)
    df['PO'] = NA_VALUE
    df.loc[~mask, 'PO'] = np.power(df.loc[~mask, 'P'], 2.0) - np.power(df.loc[~mask, 'PT'], 2.0)
    return df

In [64]:
# train, test = map(momentum, [train, test])

In [65]:
train = momentum(train)

In [66]:
train.loc[:, ['P', 'PT', 'PO']].head()

,P,PT,PO
0,50268.359507,2589.074895,2.520205e+09
1,9136.633410,1214.306768,8.200353e+07
2,46241.344045,4021.866299,2.122086e+09
3,14855.922836,2900.460154,2.122858e+08
4,21442.937297,1915.277178,4.561313e+08


### Отношение NShared к FOI_hits_N

In [67]:
def nshared_foihits(df):
    df.loc[:, "frac_NShared_FOI_hits_N"] = df.loc[:, "NShared"] / df.loc[:, "FOI_hits_N"] 
    df.loc[:, "frac_NShared_FOI_hits_N"] = df.loc[:, "frac_NShared_FOI_hits_N"].fillna(-1)
    return df

In [68]:
# train, test = map(nshared_foihits, [train, test])

In [69]:
train = nshared_foihits(train)

In [70]:
train.loc[:, ["NShared", "FOI_hits_N", "frac_NShared_FOI_hits_N"]].head()

,NShared,FOI_hits_N,frac_NShared_FOI_hits_N
0,0,5,0.0
1,0,8,0.0
2,0,5,0.0
3,0,5,0.0
4,0,9,0.0


### Усредняем дисперсии

In [71]:
def mean_std(df):
    for station in range(4):
        cols = [f'MatchedHit_D{axis}[{station}]' for axis in 'XYZ']
        mask = np.isclose(df.loc[:, cols], -1).any(axis=1)
        
        name = f'MatchedHit_D[{station}]_mean'
        df[name] = -1
        df.loc[~mask, name] = df.loc[~mask, cols].mean(axis=1)
        
    for station in range(4):
        cols = [f'Mextra_D{axis}2[{station}]' for axis in 'XY']
        mask = np.isclose(df.loc[:, cols], -1).any(axis=1)
        
        name = f'Mextra_D2[{station}]_mean'
        df[name] = -1
        df.loc[~mask, name] = df.loc[~mask, cols].mean(axis=1)

    return df

In [72]:
# train, test = map(mean_std, [train, test])

In [73]:
train = mean_std(train)

### Удаляем лишние колонки

In [74]:
import operator
from functools import reduce

def drop_columns(df):
    mask = [df.columns.str.startswith('MatchedHit_X'),
            df.columns.str.startswith('MatchedHit_Y'),
            df.columns.str.startswith('Lextra_X'),
            df.columns.str.startswith('Lextra_Y'),
            df.columns.str.startswith('FOI_hits_N['),
            df.columns == "ndof",
            df.columns.str.startswith('MatchedHit_DX') & ~df.columns.str.endswith('_mean'),
            df.columns.str.startswith('MatchedHit_DY') & ~df.columns.str.endswith('_mean'),
            df.columns.str.startswith('MatchedHit_DZ') & ~df.columns.str.endswith('_mean'),
            df.columns.str.startswith('Mextra_D') & ~df.columns.str.endswith('_mean')]
    mask = reduce(operator.or_, mask)
    df.drop(columns=df.columns[mask], inplace=True)
    return df

In [75]:
# train, test = map(drop_columns, [train, test])

In [76]:
train = drop_columns(train)

## Предсказание модели

I know this is incorrect. See it as a low-hanging fruit to beat the baseline. Catboost affirmatively [refused](https://github.com/catboost/catboost/pull/399) to support negative weights. At the same time, its evaluation is [extremly fast](https://catboost.ai/news/best-in-class-inference-and-a-ton-of-speedups), so we are using it as the timing benchmark for Track 2. Feel free to use a patched version with disabled negative weights check.

In [77]:
features = list(filter(lambda s: s not in utils.TRAIN_COLUMNS, train.columns))
features

['ncl[0]',
 'ncl[1]',
 'ncl[2]',
 'ncl[3]',
 'avg_cs[0]',
 'avg_cs[1]',
 'avg_cs[2]',
 'avg_cs[3]',
 'MatchedHit_TYPE[0]',
 'MatchedHit_TYPE[1]',
 'MatchedHit_TYPE[2]',
 'MatchedHit_TYPE[3]',
 'MatchedHit_Z[0]',
 'MatchedHit_Z[1]',
 'MatchedHit_Z[2]',
 'MatchedHit_Z[3]',
 'MatchedHit_T[0]',
 'MatchedHit_T[1]',
 'MatchedHit_T[2]',
 'MatchedHit_T[3]',
 'MatchedHit_DT[0]',
 'MatchedHit_DT[1]',
 'MatchedHit_DT[2]',
 'MatchedHit_DT[3]',
 'NShared',
 'FOI_hits_N',
 'PT',
 'P',
 'find_closest_hit_per_station_0',
 'find_closest_hit_per_station_1',
 'find_closest_hit_per_station_2',
 'find_closest_hit_per_station_3',
 'find_closest_hit_per_station_4',
 'find_closest_hit_per_station_5',
 'find_closest_hit_per_station_6',
 'find_closest_hit_per_station_7',
 'find_closest_hit_per_station_8',
 'find_closest_hit_per_station_9',
 'find_closest_hit_per_station_10',
 'find_closest_hit_per_station_11',
 'find_closest_hit_per_station_12',
 'find_closest_hit_per_station_13',
 'find_closest_hit_per_station

In [78]:
test = train

In [79]:
models_g = [catboost.CatBoostClassifier().load_model(f"models/catboost_model_{i}.cbm") for i in range(5)]

In [80]:
predictions = [model.predict_proba(test.loc[:, features].values)[:, 1] for model in models_g]
predictions = np.asarray(predictions).T

In [81]:
df_pred = pd.DataFrame(predictions,
                       columns=["catboost_model_{}".format(i) for i in range(predictions.shape[1])],
                       index=test.index)
df_pred.to_csv("preds/private_test_catboost_preds.csv", sep=',', index=False)
df_pred.head()

,catboost_model_0,catboost_model_1,catboost_model_2,catboost_model_3,catboost_model_4
0,0.672431,0.669141,0.657438,0.641826,0.771613
1,0.832719,0.845477,0.821077,0.860323,0.861051
2,0.408567,0.357850,0.406442,0.313398,0.332586
3,0.613057,0.681692,0.734121,0.620620,0.798437
4,0.485396,0.533970,0.516588,0.523012,0.552813
